# **BEIR: A Heterogenous benchmark for Zero-shot Evaluation of Information Retrieval models** 

This notebook contains an simple and easy examples to evaluate retrieval models from our new benchmark.

## Introduction
The BEIR benchmark contains 9 diverse retrieval tasks including 17 diverse datasets. We evaluate 9 state-of-the-art retriever models all in a zero-shot evaluation setup. Today, in this colab notebook, we first will show how to download and load the 14 open-sourced datasets with just three lines of code. Afterward, we would load some state-of-the-art dense retrievers (bi-encoders) such as SBERT, ANCE, DPR models and use them for retrieval and evaluate them in a zero-shot setup.

Don't hesitate to send us an e-mail or report an issue, if something is broken (and it shouldn't be) or if you have further questions.

Developed by Nandan Thakur, Researcher @ UKP Lab, TU Darmstadt

(https://nthakur.xyz) (nandant@gmail.com)

In [1]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [2]:
# Install the beir PyPI package
!pip install beir

In [3]:
from beir import util, LoggingHandler

import logging
import pathlib, os

#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout

/Users/A860312/opt/anaconda3/lib/python3.9/site-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# **BEIR Datasets**

BEIR contains 17 diverse datasets overall. You can view all the datasets (14 downloadable) with the link below:

[``https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/``](https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/)

Please refer GitHub page to evaluate on other datasets (3 of them).


We include the following datasets in BEIR:

| Dataset   | Website| BEIR-Name | Domain     | Relevancy| Queries  | Documents | Avg. Docs/Q | Download | 
| -------- | -----| ---------| ----------- | ---------| ---------| --------- | ------| ------------| 
| MSMARCO    | [``Homepage``](https://microsoft.github.io/msmarco/)| ``msmarco`` | Misc.       |  Binary  |  6,980   |  8.84M     |    1.1 | Yes |  
| TREC-COVID |  [``Homepage``](https://ir.nist.gov/covidSubmit/index.html)| ``trec-covid``| Bio-Medical |  3-level|50|  171K| 493.5 | Yes | 
| NFCorpus   | [``Homepage``](https://www.cl.uni-heidelberg.de/statnlpgroup/nfcorpus/) | ``nfcorpus``  | Bio-Medical |  3-level |  323     |  3.6K     |  38.2 | Yes |
| BioASQ     | [``Homepage``](http://bioasq.org) | ``bioasq``| Bio-Medical |  Binary  |   500    |  14.91M    |  8.05 | No | 
| NQ         | [``Homepage``](https://ai.google.com/research/NaturalQuestions) | ``nq``| Wikipedia   |  Binary  |  3,452   |  2.68M  |  1.2 | Yes | 
| HotpotQA   | [``Homepage``](https://hotpotqa.github.io) | ``hotpotqa``| Wikipedia   |  Binary  |  7,405   |  5.23M  |  2.0 | Yes |
| FiQA-2018  | [``Homepage``](https://sites.google.com/view/fiqa/) | ``fiqa``    | Finance     |  Binary  |  648     |  57K    |  2.6 | Yes | 
| Signal-1M (RT) | [``Homepage``](https://research.signal-ai.com/datasets/signal1m-tweetir.html)| ``signal1m`` | Twitter     |  3-level  |   97   |  2.86M  |  19.6 | No |
| TREC-NEWS  | [``Homepage``](https://trec.nist.gov/data/news2019.html) | ``trec-news``    | News     |  5-level  |   57    |  595K    |  19.6 | No |
| ArguAna    | [``Homepage``](http://argumentation.bplaced.net/arguana/data) | ``arguana`` | Misc.       |  Binary  |  1,406     |  8.67K    |  1.0 | Yes |
| Touche-2020| [``Homepage``](https://webis.de/events/touche-20/shared-task-1.html) | ``webis-touche2020``| Misc.       |  6-level  |  49     |  382K    |  49.2 |  Yes |
| CQADupstack| [``Homepage``](http://nlp.cis.unimelb.edu.au/resources/cqadupstack/) | ``cqadupstack``| StackEx.      |  Binary  |  13,145 |  457K  |  1.4 |  Yes |
| Quora| [``Homepage``](https://www.quora.com/q/quoradata/First-Quora-Dataset-Release-Question-Pairs) | ``quora``| Quora  | Binary  |  10,000     |  523K    |  1.6 |  Yes | 
| DBPedia | [``Homepage``](https://github.com/iai-group/DBpedia-Entity/) | ``dbpedia-entity``| Wikipedia |  3-level  |  400    |  4.63M    |  38.2 |  Yes | 
| SCIDOCS| [``Homepage``](https://allenai.org/data/scidocs) | ``scidocs``| Scientific |  Binary  |  1,000     |  25K    |  4.9 |  Yes | 
| FEVER| [``Homepage``](http://fever.ai) | ``fever``| Wikipedia     |  Binary  |  6,666     |  5.42M    |  1.2|  Yes | 
| Climate-FEVER| [``Homepage``](http://climatefever.ai) | ``climate-fever``| Wikipedia |  Binary  |  1,535     |  5.42M |  3.0 |  Yes |
| SciFact| [``Homepage``](https://github.com/allenai/scifact) | ``scifact``| Scientific |  Binary  |  300     |  5K    |  1.1 |  Yes |


For Simplicity, we will show example with the one of the smallest datasets - ``SciFact`` for our example. 

You can evaluate any dataset you wish by looking at the table above.

In [4]:
import pathlib, os
from beir import util

dataset = "scifact"
url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset)
out_dir = os.path.join(os.getcwd(), "datasets")
data_path = util.download_and_unzip(url, out_dir)
print("Dataset downloaded here: {}".format(data_path))

Dataset downloaded here: /Users/A860312/phrase_extraction/datasets/scifact


# **Folder Structure of any BEIR dataset**

* scifact/
    * corpus.jsonl 
    * queries.jsonl 
    * qrels/
        * train.tsv
        * dev.tsv
        * test.tsv

In [5]:
!ls datasets/scifact/

corpus.jsonl  qrels         queries.jsonl


# **Data Loading**

In [6]:
from beir.datasets.data_loader import GenericDataLoader

data_path = "datasets/scifact"
corpus, queries, qrels = GenericDataLoader(data_path).load(split="test") # or split = "train" or "dev"

2022-08-11 00:07:59 - Loading Corpus...


  0%|          | 0/5183 [00:00<?, ?it/s]

2022-08-11 00:07:59 - Loaded 5183 TEST Documents.
2022-08-11 00:07:59 - Doc Example: {'text': 'Alterations of the architecture of cerebral white matter in the developing human brain can affect cortical development and result in functional disabilities. A line scan diffusion-weighted magnetic resonance imaging (MRI) sequence with diffusion tensor analysis was applied to measure the apparent diffusion coefficient, to calculate relative anisotropy, and to delineate three-dimensional fiber architecture in cerebral white matter in preterm (n = 17) and full-term infants (n = 7). To assess effects of prematurity on cerebral white matter development, early gestation preterm infants (n = 10) were studied a second time at term. In the central white matter the mean apparent diffusion coefficient at 28 wk was high, 1.8 microm2/ms, and decreased toward term to 1.2 microm2/ms. In the posterior limb of the internal capsule, the mean apparent diffusion coefficients at both times were similar (1.2 vers

In [7]:
qrels

{'1': {'31715818': 1},
 '3': {'14717500': 1},
 '5': {'13734012': 1},
 '13': {'1606628': 1},
 '36': {'5152028': 1, '11705328': 1},
 '42': {'18174210': 1},
 '48': {'13734012': 1},
 '49': {'5953485': 1},
 '50': {'12580014': 1},
 '51': {'45638119': 1},
 '53': {'45638119': 1},
 '54': {'49556906': 1},
 '56': {'4709641': 1},
 '57': {'4709641': 1},
 '70': {'5956380': 1, '4414547': 1},
 '72': {'6076903': 1},
 '75': {'4387784': 1},
 '94': {'1215116': 1},
 '99': {'18810195': 1},
 '100': {'4381486': 1},
 '113': {'6157837': 1},
 '115': {'33872649': 1},
 '118': {'6372244': 1},
 '124': {'4883040': 1},
 '127': {'21598000': 1},
 '128': {'8290953': 1},
 '129': {'27768226': 1},
 '130': {'27768226': 1},
 '132': {'7975937': 1},
 '133': {'38485364': 1,
  '6969753': 1,
  '17934082': 1,
  '16280642': 1,
  '12640810': 1},
 '137': {'26016929': 1},
 '141': {'6955746': 1, '14437255': 1},
 '142': {'10582939': 1},
 '143': {'10582939': 1},
 '146': {'10582939': 1},
 '148': {'1084345': 1},
 '163': {'18872233': 1},
 '1

In [8]:
queries

{'1': '0-dimensional biomaterials show inductive properties.',
 '3': '1,000 genomes project enables mapping of genetic sequence variation consisting of rare variants with larger penetrance effects than common variants.',
 '5': '1/2000 in UK have abnormal PrP positivity.',
 '13': '5% of perinatal mortality is due to low birth weight.',
 '36': 'A deficiency of vitamin B12 increases blood levels of homocysteine.',
 '42': 'A high microerythrocyte count raises vulnerability to severe anemia in homozygous alpha (+)- thalassemia trait subjects.',
 '48': 'A total of 1,000 people in the UK are asymptomatic carriers of vCJD infection.',
 '49': 'ADAR1 binds to Dicer to cleave pre-miRNA.',
 '50': 'AIRE is expressed in some skin tumors.',
 '51': 'ALDH1 expression is associated with better breast cancer outcomes.',
 '53': 'ALDH1 expression is associated with poorer prognosis in breast cancer.',
 '54': 'AMP-activated protein kinase (AMPK) activation increases inflammation-related fibrosis in the lung

AttributeError: 'dict' object has no attribute 'to_dict'

In [8]:
for k,v in queries.items():
    print(k ,"-->", v)


1 --> 0-dimensional biomaterials show inductive properties.
3 --> 1,000 genomes project enables mapping of genetic sequence variation consisting of rare variants with larger penetrance effects than common variants.
5 --> 1/2000 in UK have abnormal PrP positivity.
13 --> 5% of perinatal mortality is due to low birth weight.
36 --> A deficiency of vitamin B12 increases blood levels of homocysteine.
42 --> A high microerythrocyte count raises vulnerability to severe anemia in homozygous alpha (+)- thalassemia trait subjects.
48 --> A total of 1,000 people in the UK are asymptomatic carriers of vCJD infection.
49 --> ADAR1 binds to Dicer to cleave pre-miRNA.
50 --> AIRE is expressed in some skin tumors.
51 --> ALDH1 expression is associated with better breast cancer outcomes.
53 --> ALDH1 expression is associated with poorer prognosis in breast cancer.
54 --> AMP-activated protein kinase (AMPK) activation increases inflammation-related fibrosis in the lungs.
56 --> APOE4 expression in iPSC

# **Dense Retrieval using Exact Search**

## **Sentence-BERT**
We use the [``distilbert-base-msmarco-v3``](https://www.sbert.net/docs/pretrained-models/msmarco-v3.html) SBERT model in this example.

In [9]:
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval import models
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES

#### Dense Retrieval using SBERT (Sentence-BERT) ####
#### Provide any pretrained sentence-transformers model
#### The model was fine-tuned using cosine-similarity.
#### Complete list - https://www.sbert.net/docs/pretrained_models.html

model = DRES(models.SentenceBERT("msmarco-distilbert-base-v3"), batch_size=128)
retriever = EvaluateRetrieval(model, score_function="cos_sim")

#### Retrieve dense results (format of results is identical to qrels)
results = retriever.retrieve(corpus, queries)

2022-08-09 21:46:35 - Loading faiss with AVX2 support.
2022-08-09 21:46:35 - Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
2022-08-09 21:46:35 - Loading faiss.
2022-08-09 21:46:35 - Successfully loaded faiss.


ModuleNotFoundError: No module named 'fusion_search'

In [8]:
print(type(results))

<class 'dict'>


In [11]:
cnt = -1
for k,v in results.items():
    print (k, '-->', v)
    cnt = cnt + 1
    if ( cnt >= 5):
        break;


1 --> {'4346436': 0.38071900606155396, '16736872': 0.3462148606777191, '33986200': 0.3404478430747986, '10017612': 0.3402380049228668, '16962732': 0.31940752267837524, '935034': 0.3118133842945099, '14717213': 0.30876457691192627, '24923605': 0.3085263967514038, '2158516': 0.3065412640571594, '121581019': 0.30591726303100586, '19685306': 0.3050881028175354}
3 --> {'11117679': 0.5263203382492065, '19058822': 0.5145615339279175, '23389795': 0.49923810362815857, '2739854': 0.49898457527160645, '4378885': 0.49221664667129517, '4632921': 0.4869627356529236, '1388704': 0.4809066653251648, '2359152': 0.47881975769996643, '16172576': 0.471762478351593, '4414547': 0.47001296281814575, '7020505': 0.4696445167064667}
5 --> {'12881593': 0.40512338280677795, '25261168': 0.4017601013183594, '76415938': 0.40041297674179077, '23124332': 0.399053156375885, '9764256': 0.3945937156677246, '5433667': 0.39233773946762085, '9272598': 0.3863191604614258, '21550246': 0.3842666447162628, '1583041': 0.377422630

In [ ]:
#### Evaluate your retrieval using NDCG@k, MAP@K ...

logging.info("Retriever evaluation for k in: {}".format(retriever.k_values))
ndcg, _map, recall, precision = retriever.evaluate(qrels, results, retriever.k_values)

2021-04-20 15:46:44 - Retriever evaluation for k in: [1, 3, 5, 10, 100, 1000]
2021-04-20 15:46:44 - 

2021-04-20 15:46:44 - NDCG@1: 0.4233
2021-04-20 15:46:44 - NDCG@3: 0.4842
2021-04-20 15:46:44 - NDCG@5: 0.5104
2021-04-20 15:46:44 - NDCG@10: 0.5379
2021-04-20 15:46:44 - NDCG@100: 0.5759
2021-04-20 15:46:44 - NDCG@1000: 0.5913
2021-04-20 15:46:44 - 

2021-04-20 15:46:44 - MAP@1: 0.3994
2021-04-20 15:46:44 - MAP@3: 0.4593
2021-04-20 15:46:44 - MAP@5: 0.4768
2021-04-20 15:46:44 - MAP@10: 0.4889
2021-04-20 15:46:44 - MAP@100: 0.4974
2021-04-20 15:46:44 - MAP@1000: 0.4980
2021-04-20 15:46:44 - 

2021-04-20 15:46:44 - Recall@1: 0.3994
2021-04-20 15:46:44 - Recall@3: 0.5256
2021-04-20 15:46:44 - Recall@5: 0.5887
2021-04-20 15:46:44 - Recall@10: 0.6723
2021-04-20 15:46:44 - Recall@100: 0.8460
2021-04-20 15:46:44 - Recall@1000: 0.9683
2021-04-20 15:46:44 - 

2021-04-20 15:46:44 - P@1: 0.4233
2021-04-20 15:46:44 - P@3: 0.1933
2021-04-20 15:46:44 - P@5: 0.1333
2021-04-20 15:46:44 - P@10: 0.0757

In [ ]:
import random

#### Print top-k documents retrieved ####
top_k = 10

query_id, ranking_scores = random.choice(list(results.items()))
scores_sorted = sorted(ranking_scores.items(), key=lambda item: item[1], reverse=True)
logging.info("Query : %s\n" % queries[query_id])

for rank in range(top_k):
    doc_id = scores_sorted[rank][0]
    # Format: Rank x: ID [Title] Body
    logging.info("Rank %d: %s [%s] - %s\n" % (rank+1, doc_id, corpus[doc_id].get("title"), corpus[doc_id].get("text")))

2021-04-20 15:57:32 - Query : Rapid up-regulation and higher basal expression of interferon-induced genes increase survival of granule cell neurons that are infected by West Nile virus.

2021-04-20 15:57:32 - Rank 1: 7717468 [Loss of allergen 1 confers a hypervirulent phenotype that resembles mucoid switch variants of Cryptococcus neoformans.] - Microbial survival in a host is usually dependent on the ability of a pathogen to undergo changes that promote escape from host defense mechanisms. The human-pathogenic fungus Cryptococcus neoformans undergoes phenotypic switching in vivo that promotes persistence in tissue. By microarray and real-time PCR analyses, the allergen 1 gene (ALL1) was found to be downregulated in the hypervirulent mucoid switch variant, both during logarithmic growth and during intracellular growth in macrophages. The ALL1 gene encodes a small cytoplasmic protein that is involved in capsule formation. Growth of an all1Delta gene deletion mutant was normal. Similar t

## **ANCE**

We use the [``msmarco-roberta-base-ance-fristp``](https://www.sbert.net/docs/pretrained-models/msmarco-v3.html) ANCE model which was fine-tuned on MSMARCO dataset for 600K steps.

In [ ]:
#### Dense Retrieval using ANCE #### 
# https://www.sbert.net/docs/pretrained-models/msmarco-v3.html
# MSMARCO Dev Passage Retrieval ANCE(FirstP) 600K model from ANCE.
# The ANCE model was fine-tuned using dot-product (dot) function.

model = DRES(models.SentenceBERT("msmarco-roberta-base-ance-fristp"))
retriever = EvaluateRetrieval(model, score_function="dot")

#### Retrieve dense results (format of results is identical to qrels)
results = retriever.retrieve(corpus, queries)

2021-04-20 15:53:21 - Load pretrained SentenceTransformer: msmarco-roberta-base-ance-fristp
2021-04-20 15:53:21 - Did not find folder msmarco-roberta-base-ance-fristp
2021-04-20 15:53:21 - Search model on server: http://sbert.net/models/msmarco-roberta-base-ance-fristp.zip
2021-04-20 15:53:21 - Downloading sentence transformer model from http://sbert.net/models/msmarco-roberta-base-ance-fristp.zip and saving it at /root/.cache/torch/sentence_transformers/sbert.net_models_msmarco-roberta-base-ance-fristp



2021-04-20 15:53:44 - Load SentenceTransformer from folder: /root/.cache/torch/sentence_transformers/sbert.net_models_msmarco-roberta-base-ance-fristp
2021-04-20 15:53:47 - Use pytorch device: cuda
2021-04-20 15:53:47 - Encoding Queries...



2021-04-20 15:53:48 - Encoding Corpus in batches... Warning: This might take a while!
2021-04-20 15:53:48 - Encoding Batch 1/1...


In [ ]:
#### Evaluate your retrieval using NDCG@k, MAP@K ...

logging.info("Retriever evaluation for k in: {}".format(retriever.k_values))
ndcg, _map, recall, precision = retriever.evaluate(qrels, results, retriever.k_values)

2021-04-20 15:57:42 - Retriever evaluation for k in: [1, 3, 5, 10, 100, 1000]
2021-04-20 15:57:42 - 

2021-04-20 15:57:42 - NDCG@1: 0.4100
2021-04-20 15:57:42 - NDCG@3: 0.4682
2021-04-20 15:57:42 - NDCG@5: 0.4871
2021-04-20 15:57:42 - NDCG@10: 0.5114
2021-04-20 15:57:42 - NDCG@100: 0.5493
2021-04-20 15:57:42 - NDCG@1000: 0.5665
2021-04-20 15:57:42 - 

2021-04-20 15:57:42 - MAP@1: 0.3843
2021-04-20 15:57:42 - MAP@3: 0.4416
2021-04-20 15:57:42 - MAP@5: 0.4546
2021-04-20 15:57:42 - MAP@10: 0.4661
2021-04-20 15:57:42 - MAP@100: 0.4732
2021-04-20 15:57:42 - MAP@1000: 0.4738
2021-04-20 15:57:42 - 

2021-04-20 15:57:42 - Recall@1: 0.3843
2021-04-20 15:57:42 - Recall@3: 0.5124
2021-04-20 15:57:42 - Recall@5: 0.5613
2021-04-20 15:57:42 - Recall@10: 0.6333
2021-04-20 15:57:42 - Recall@100: 0.8187
2021-04-20 15:57:42 - Recall@1000: 0.9567
2021-04-20 15:57:42 - 

2021-04-20 15:57:42 - P@1: 0.4100
2021-04-20 15:57:42 - P@3: 0.1889
2021-04-20 15:57:42 - P@5: 0.1260
2021-04-20 15:57:42 - P@10: 0.0720

# **Lexical Retrieval using BM25 (Elasticsearch)**

## 1. Download and setup the Elasticsearch instance
Reference: https://colab.research.google.com/github/tensorflow/io/blob/master/docs/tutorials/elasticsearch.ipynb

For demo purposes, the open-source version of the elasticsearch package is used.

In [ ]:
%%bash

wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512
tar -xzf elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
sudo chown -R daemon:daemon elasticsearch-7.9.2/
shasum -a 512 -c elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512 

elasticsearch-oss-7.9.2-linux-x86_64.tar.gz: OK


Run the instance as a daemon process


In [ ]:
%%bash --bg

sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch

Starting job # 0 in a separate thread.


In [ ]:
import time

# Sleep for few seconds to let the instance start.
time.sleep(20)

Once the instance has been started, grep for ``elasticsearch`` in the processes list to confirm the availability.

In [ ]:
%%bash

ps -ef | grep elasticsearch

root         664     662  0 16:05 ?        00:00:00 sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch
daemon       665     664 23 16:05 ?        00:00:16 /content/elasticsearch-7.9.2/jdk/bin/java -Xshare:auto -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -XX:+ShowCodeDetailsInExceptionMessages -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dio.netty.allocator.numDirectArenas=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Djava.locale.providers=SPI,COMPAT -Xms1g -Xmx1g -XX:+UseG1GC -XX:G1ReservePercent=25 -XX:InitiatingHeapOccupancyPercent=30 -Djava.io.tmpdir=/tmp/elasticsearch-1430391873797249845 -XX:+HeapDumpOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,pid,tags:filecou

In [ ]:
%%bash

curl -sX GET "localhost:9200/"

{
  "name" : "c0f3c0cd00cc",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "RjQ4yenrQomacpABczxITQ",
  "version" : {
    "number" : "7.9.2",
    "build_flavor" : "oss",
    "build_type" : "tar",
    "build_hash" : "d34da0ea4a966c4e49417f2da2f244e3e97b4e6e",
    "build_date" : "2020-09-23T00:45:33.626720Z",
    "build_snapshot" : false,
    "lucene_version" : "8.6.2",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [ ]:
from beir.retrieval.search.lexical import BM25Search as BM25
from beir.retrieval.evaluation import EvaluateRetrieval

#### Provide parameters for elastic-search
hostname = "localhost" 
index_name = "scifact" 
initialize = True # True, will delete existing index with same name and reindex all documents

model = BM25(index_name=index_name, hostname=hostname, initialize=initialize)
retriever = EvaluateRetrieval(model)

#### Retrieve dense results (format of results is identical to qrels)
results = retriever.retrieve(corpus, queries)

2021-04-20 16:09:45 - Activating Elasticsearch....
2021-04-20 16:09:45 - Elastic Search Credentials: {'hostname': 'localhost', 'index_name': 'scifact', 'keys': {'title': 'title', 'body': 'txt'}, 'timeout': 100, 'retry_on_timeout': True, 'maxsize': 24}
2021-04-20 16:09:45 - Deleting previous Elasticsearch-Index named - scifact
2021-04-20 16:09:45 - Creating fresh Elasticsearch-Index named - scifact


que: 100%|██████████| 3/3 [00:14<00:00,  4.94s/it]


In [ ]:
#### Evaluate your retrieval using NDCG@k, MAP@K ...
ndcg, _map, recall, precision = retriever.evaluate(qrels, results, retriever.k_values)

2021-04-20 16:10:20 - 

2021-04-20 16:10:20 - NDCG@1: 0.5400
2021-04-20 16:10:20 - NDCG@3: 0.6104
2021-04-20 16:10:20 - NDCG@5: 0.6297
2021-04-20 16:10:20 - NDCG@10: 0.6495
2021-04-20 16:10:20 - NDCG@100: 0.6749
2021-04-20 16:10:20 - NDCG@1000: 0.6850
2021-04-20 16:10:20 - 

2021-04-20 16:10:20 - MAP@1: 0.5176
2021-04-20 16:10:20 - MAP@3: 0.5851
2021-04-20 16:10:20 - MAP@5: 0.5974
2021-04-20 16:10:20 - MAP@10: 0.6068
2021-04-20 16:10:20 - MAP@100: 0.6130
2021-04-20 16:10:20 - MAP@1000: 0.6134
2021-04-20 16:10:20 - 

2021-04-20 16:10:20 - Recall@1: 0.5176
2021-04-20 16:10:20 - Recall@3: 0.6607
2021-04-20 16:10:20 - Recall@5: 0.7071
2021-04-20 16:10:20 - Recall@10: 0.7657
2021-04-20 16:10:20 - Recall@100: 0.8742
2021-04-20 16:10:20 - Recall@1000: 0.9550
2021-04-20 16:10:20 - 

2021-04-20 16:10:20 - P@1: 0.5400
2021-04-20 16:10:20 - P@3: 0.2356
2021-04-20 16:10:20 - P@5: 0.1547
2021-04-20 16:10:20 - P@10: 0.0840
2021-04-20 16:10:20 - P@100: 0.0099
2021-04-20 16:10:20 - P@1000: 0.0011


# **Reranking BM25 using Cross-Encoder**

In this example, we rerank the top-20 documents retrieved from BM25, using ([cross-encoder/ms-marco-electra-base](https://www.sbert.net/docs/pretrained-models/ce-msmarco.html)) SBERT cross-encoder model 

In [ ]:
from beir.reranking.models import CrossEncoder
from beir.reranking import Rerank

#### Reranking using Cross-Encoder models (list: )
cross_encoder_model = CrossEncoder('cross-encoder/ms-marco-electra-base')
reranker = Rerank(cross_encoder_model, batch_size=128)

# Rerank top-100 results using the reranker provided
rerank_results = reranker.rerank(corpus, queries, results, top_k=20)

2021-04-20 16:14:39 - Use pytorch device: cuda
2021-04-20 16:14:39 - Starting To Rerank Top-20....


In [ ]:
#### Evaluate your retrieval using NDCG@k, MAP@K ...
ndcg, _map, recall, precision = EvaluateRetrieval.evaluate(qrels, rerank_results, retriever.k_values)

2021-04-20 16:19:08 - 

2021-04-20 16:19:08 - NDCG@1: 0.5733
2021-04-20 16:19:08 - NDCG@3: 0.6314
2021-04-20 16:19:08 - NDCG@5: 0.6520
2021-04-20 16:19:08 - NDCG@10: 0.6720
2021-04-20 16:19:08 - NDCG@100: 0.6780
2021-04-20 16:19:08 - NDCG@1000: 0.6780
2021-04-20 16:19:08 - 

2021-04-20 16:19:08 - MAP@1: 0.5451
2021-04-20 16:19:08 - MAP@3: 0.6074
2021-04-20 16:19:08 - MAP@5: 0.6216
2021-04-20 16:19:08 - MAP@10: 0.6307
2021-04-20 16:19:08 - MAP@100: 0.6324
2021-04-20 16:19:08 - MAP@1000: 0.6324
2021-04-20 16:19:08 - 

2021-04-20 16:19:08 - Recall@1: 0.5451
2021-04-20 16:19:08 - Recall@3: 0.6758
2021-04-20 16:19:08 - Recall@5: 0.7260
2021-04-20 16:19:08 - Recall@10: 0.7844
2021-04-20 16:19:08 - Recall@100: 0.8078
2021-04-20 16:19:08 - Recall@1000: 0.8078
2021-04-20 16:19:08 - 

2021-04-20 16:19:08 - P@1: 0.5733
2021-04-20 16:19:08 - P@3: 0.2444
2021-04-20 16:19:08 - P@5: 0.1613
2021-04-20 16:19:08 - P@10: 0.0880
2021-04-20 16:19:08 - P@100: 0.0090
2021-04-20 16:19:08 - P@1000: 0.0009
